In [54]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text, MetaData, Table, Column, String, Integer, ForeignKey, Numeric, Date

## Extracción

* Leer el archivo csv

In [9]:
df = pd.read_csv("data_prueba_tecnica.csv")
df

,id,name,company_id,amount,status,created_at,paid_at
0,48ba4bdbfb56ceebb32f2bd0263e759be942af3d,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,3.00,voided,2019-03-19,NaN
1,05fc6f5ac66b6ee7e4253aa5d0c2299eb47aaaf4,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,3.00,pending_payment,2019-05-06,NaN
2,2cdce231c1fc6a2061bfa2f1d978351fe217245d,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,3.00,voided,2019-02-22,NaN
3,81633ba310a50b673efd469c37139576982901aa,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,102.61,paid,2019-02-27,2019-02-27
4,6ccfc4c24e788e4bca448df343698782db6b0c0b,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,184.49,paid,2019-02-05,2019-02-05
...,...,...,...,...,...,...,...
9995,4907c36c037bfa4874047e14f722329016dc3908,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,59.88,paid,2019-03-14,2019-03-14
9996,ef9ec85887418d2ccf220686312cfc4c615665d5,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,23.05,paid,2019-02-20,2019-02-20
9997,50bb3950ecde94ca3e07853003a11a413508da36,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,20.50,paid,2019-03-19,2019-03-19
9998,1b2c4e503a4a952064b70369cf8e72937c720474,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,81.42,paid,2019-03-10,2019-03-10


**Verificar si hay valores nulos:**

*   Existen valores nulos en la columna id, name y company_id

In [10]:
df.isna().sum()

id               3
name             3
company_id       4
amount           0
status           0
created_at       0
paid_at       3991
dtype: int64

## Limpieza de datos:

1.   Eliminé la columna id, si esta contenía valores nulos, ya que la columna id, no debe ser un valor único para cada registro y no puede aceptar valores nulos.

*    Para no afectar el dataframe original, lo asigné a la variable df_clean*

In [11]:
if df['id'].isna().any():
    df_clean = df.drop(columns=['id'])

df_clean.reset_index(drop=True)
df_clean.index.name = 'id'

df_clean

,name,company_id,amount,status,created_at,paid_at
id,,,,,,
0,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,3.00,voided,2019-03-19,NaN
1,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,3.00,pending_payment,2019-05-06,NaN
2,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,3.00,voided,2019-02-22,NaN
3,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,102.61,paid,2019-02-27,2019-02-27
4,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,184.49,paid,2019-02-05,2019-02-05
...,...,...,...,...,...,...
9995,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,59.88,paid,2019-03-14,2019-03-14
9996,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,23.05,paid,2019-02-20,2019-02-20
9997,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,20.50,paid,2019-03-19,2019-03-19


Dentro de la limpieza de datos vi la relación entre las compañías y su id (quitando los duplicados)

*   Observé que hay compañías que tienen el mismo id pero en ciertos nombres por alguna razón el dato no se extrajo bien
* También existían compañías que si estaban bien escritas pero el id era Nan o ***

In [12]:
companies_ids = df[['company_id', 'name']].drop_duplicates()
companies_ids

,company_id,name
id,,
0,cbf1c8b09cd5b549416d49d220a40cbd317f952e,MiPasajefy
78,8f642dc67fccf861548dfe1c761ce22f795e91f0,Muebles chidos
262,NaN,MiPasajefy
603,*******,MiPasajefy
731,cbf1c8b09cd5b549416d49d220a40cbd317f952e,NaN
1320,cbf1c8b09cd5b549416d49d220a40cbd317f952e,MiPas0xFFFF
1479,cbf1c8b09cd5b549416d49d220a40cbd317f952e,MiP0xFFFF


**Identificación y relación entre company_id y name:**

1.   Para la columna name, si su company_id era uno de los definidos dentro del diccionario dicc_company_id, se colocaría el nombre correcto correspondiente a ese id.
2.   Para la columna company_id, si su name era una de las definidas dentro del diccionario dicc_company_name, se colocaría el id correcpondiente a ese nombre de compañía.

In [13]:
dicc_company_id = {
    'cbf1c8b09cd5b549416d49d220a40cbd317f952e': 'MiPasajefy',
    '8f642dc67fccf861548dfe1c761ce22f795e91f0': 'Muebles chidos'
}
dicc_company_name = {
    'MiPasajefy': 'cbf1c8b09cd5b549416d49d220a40cbd317f952e',
    'Muebles chidos': '8f642dc67fccf861548dfe1c761ce22f795e91f0',
}

df_clean['name'] = np.where(
    df['company_id'].isin(dicc_company_id.keys()),
    df['company_id'].map(dicc_company_id),
    df['name']
)

df_clean['company_id'] = np.where(
    df['name'].isin(dicc_company_name.keys()),
    df['name'].map(dicc_company_name),
    df['company_id']
)

df_clean

,name,company_id,amount,status,created_at,paid_at
id,,,,,,
0,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,3.00,voided,2019-03-19,NaN
1,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,3.00,pending_payment,2019-05-06,NaN
2,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,3.00,voided,2019-02-22,NaN
3,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,102.61,paid,2019-02-27,2019-02-27
4,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,184.49,paid,2019-02-05,2019-02-05
...,...,...,...,...,...,...
9995,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,59.88,paid,2019-03-14,2019-03-14
9996,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,23.05,paid,2019-02-20,2019-02-20
9997,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,20.50,paid,2019-03-19,2019-03-19


**Verificación de la correcta relación entre compañías y sus ids:**

In [14]:
companies_ids = df_clean[['company_id', 'name']].drop_duplicates()
companies_ids

,company_id,name
id,,
0,cbf1c8b09cd5b549416d49d220a40cbd317f952e,MiPasajefy
78,8f642dc67fccf861548dfe1c761ce22f795e91f0,Muebles chidos


**Verificación de valores nulos, después de la limpieza:**

*   paid_at continua teniendo valores nulos, ya que solo tiene valor asignado aquellos que tienen estatus paid

In [15]:
df_clean.isna().sum()

name             0
company_id       0
amount           0
status           0
created_at       0
paid_at       3991
dtype: int64

**Identificamos los tipos de datos:**

In [16]:
df_clean.dtypes

name           object
company_id     object
amount        float64
status         object
created_at     object
paid_at        object
dtype: object

## Transformación
**Transformamos los datos al tipo de dato adecuado:**

In [17]:
df_clean['name'] = df_clean['name'].astype(str)
df_clean['company_id'] = df_clean['company_id'].astype(str)
df_clean['amount'] = df_clean['amount'].round(2)
df_clean['created_at'] = pd.to_datetime(df_clean['created_at'], errors='coerce')
df_clean['paid_at'] = pd.to_datetime(df_clean['paid_at'], errors='coerce')
df_clean.dtypes

name                  object
company_id            object
amount               float64
status                object
created_at    datetime64[ns]
paid_at       datetime64[ns]
dtype: object

**Hice un dataframe para las compañías:**

*Posteriormente será una tabla en la base de datos.*

In [18]:
companies = df_clean[['company_id', 'name']].drop_duplicates()
companies

,company_id,name
id,,
0,cbf1c8b09cd5b549416d49d220a40cbd317f952e,MiPasajefy
78,8f642dc67fccf861548dfe1c761ce22f795e91f0,Muebles chidos


**Hice un dataframe para los cargos:**

*Posteriormente será una tabla separada en la base de datos*

In [24]:
charges = df_clean[['company_id', 'amount', 'status', 'created_at', 'paid_at']]
charges

,company_id,amount,status,created_at,paid_at
id,,,,,
0,cbf1c8b09cd5b549416d49d220a40cbd317f952e,3.00,voided,2019-03-19,NaT
1,cbf1c8b09cd5b549416d49d220a40cbd317f952e,3.00,pending_payment,2019-05-06,NaT
2,cbf1c8b09cd5b549416d49d220a40cbd317f952e,3.00,voided,2019-02-22,NaT
3,cbf1c8b09cd5b549416d49d220a40cbd317f952e,102.61,paid,2019-02-27,2019-02-27
4,cbf1c8b09cd5b549416d49d220a40cbd317f952e,184.49,paid,2019-02-05,2019-02-05
...,...,...,...,...,...
9995,cbf1c8b09cd5b549416d49d220a40cbd317f952e,59.88,paid,2019-03-14,2019-03-14
9996,cbf1c8b09cd5b549416d49d220a40cbd317f952e,23.05,paid,2019-02-20,2019-02-20
9997,cbf1c8b09cd5b549416d49d220a40cbd317f952e,20.50,paid,2019-03-19,2019-03-19


**Respaldar los datos en archivos csv:**


In [26]:
companies.to_csv("companies.csv", index=False)

In [25]:
charges.to_csv("charges.csv", index=True)

## Dispersión
* Conexión a la base de datos, en este caso mysql

In [39]:
engine = create_engine('mysql+pymysql://root:@localhost/data')

* Crear las tablas en la bd

In [47]:
metadata = MetaData()

Table(
    "companies",
    metadata,
    Column("company_id", String(50), primary_key=True),
    Column("name", String(50), nullable=False)
)
metadata.create_all(engine)
companies.to_sql(name = "companies", con = engine, if_exists = "append", index = False)

2

In [55]:
Table(
    "charges",
    metadata,
    Column("id", Integer(), primary_key=True),
    Column("company_id", String(50), ForeignKey("companies.company_id"), nullable=False),
    Column("amount", Numeric(10,2), nullable=False),
    Column("status", String(30), nullable=False),
    Column("created_at", Date(), nullable=False),
    Column("paid_at", Date(), nullable=True)
)
metadata.create_all(engine)
charges.to_sql(name='charges', con=engine, if_exists='append', index=False)

10000

## Vista 
* Vista para ver el monto total transaccionado por día para las diferentes compañías

In [62]:
with engine.connect() as con:
    con.execute(text("""
        CREATE OR REPLACE VIEW transacciones_diarias AS
        SELECT
            c.company_id,
            c.name AS nombre,
            ch.created_at AS fecha_transaccion,
            SUM(ch.amount) AS monto_total
        FROM
            companies AS c
        JOIN 
            charges AS ch ON c.company_id = ch.company_id
        GROUP BY 
            c.company_id, c.name, ch.created_at
        ORDER BY 
            fecha_transaccion DESC, monto_total DESC
    """))
    con.commit()